# Probing part 2

Vantage points will probe the targets.

Vantage points are the anchors and the probes.  
As always, targets are the anchors.  

This notebook is an implementation of the million scale method. Check the paper for more information.  
To do after create_datasets.ipynb

In [ ]:
import logging
import uuid

from random import sample

from scripts.utils.file_utils import load_json, dump_json
from scripts.ripe_atlas.ping_and_traceroute_classes import PING
from scripts.ripe_atlas.atlas_api import get_measurements_from_tag
from default import ANCHORS_FILE, PROBES_FILE, TARGET_ALL_VP, MEASUREMENT_CONFIG_PATH, RIPE_CREDENTIALS


logging.basicConfig(level=logging.INFO)


#NB_TARGETS = 2
#NB_VPS = 4

# Comment the two lines below and uncomment the two lines above to choose the number of targets and VPs.
# The default value (-1) will use all the dataset.

NB_TARGETS = -1
NB_VPS = -1

## Load datasets

In [ ]:
anchors = load_json(ANCHORS_FILE)

probes = load_json(PROBES_FILE)

## Select targets and vps dataset

In [ ]:
if NB_TARGETS > 0:
    targets = sample(anchors, k=NB_TARGETS)
else:
    targets = anchors

for i in range(len(targets)):
    targets[i] = targets[i]["address_v4"]


probes.extend(anchors)

vps = {}

if NB_VPS > 0:
    shuffle = sample(probes, k=NB_VPS)
    for probe in shuffle:
        vps[probe["address_v4"]] = probe
else:
    shuffle = sample(probes)
    for probe in shuffle:
        vps[probe["address_v4"]] = probe


print(f"nb targets: {len(targets)}")
print(f"nb_vps : {len(vps)}")

## Probe targets
WARNING : Time consumming section

In [ ]:
dry_run = False
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all probes and all anchors as vps",
    "type": "target",
    "af": 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
config_file = MEASUREMENT_CONFIG_PATH / f"{measurement_config['UUID']}.json"
dump_json(measurement_config, config_file)

pinger = PING(RIPE_CREDENTIALS)


try:
    cached_results = load_json(TARGET_ALL_VP)
    logging.info(
        f"initial length targets: {len(targets)}, cached measurements : {len(cached_results)}"
    )

    targets = list(
        set(targets).difference(set(cached_results)))

    logging.info(
        f"after removing cached: {len(targets)}, cached measurements : {len(cached_results)}"
    )
except FileNotFoundError:
    logging.info("No cached results available")
    pass

logging.info(
        f"Starting measurements {str(measurement_uuid)} with parameters: dry_run={dry_run}; nb_targets={len(targets)}; nb_vps={len(vps)}."
    )

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = pinger.ping_by_target(
    targets=targets,
    vps=vps,
    tag=measurement_uuid,
    dry_run=dry_run
)

# save measurement configuration after measurements
dump_json(measurement_config, config_file)

## Retrieve target results
WARNING : Time consumming section

In [ ]:
measurement_results = get_measurements_from_tag(str(measurement_config["UUID"]))

# test that everything is alright
print(f"nb measurements retrieved: {len(measurement_results)}")

for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10:
        break
    print(target_addr, ":", len(results))

# save results
dump_json(measurement_results, TARGET_ALL_VP)